In [114]:
# 1. Define the problem
## Predict resale HDB property prices

In [115]:
# 2. Pre-process data
## Collect data
## Clean data
## Wrangle data

In [116]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [117]:
df_hdb = pd.read_csv("data.csv")

In [118]:
df_hdb.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [119]:
# Check dtype of features
df_hdb.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154668 entries, 0 to 154667
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                154668 non-null  object 
 1   town                 154668 non-null  object 
 2   flat_type            154668 non-null  object 
 3   block                154668 non-null  object 
 4   street_name          154668 non-null  object 
 5   storey_range         154668 non-null  object 
 6   floor_area_sqm       154668 non-null  float64
 7   flat_model           154668 non-null  object 
 8   lease_commence_date  154668 non-null  int64  
 9   remaining_lease      154668 non-null  object 
 10  resale_price         154668 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 13.0+ MB


In [120]:
# Check missing values of features
df_hdb.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64

In [121]:
# Convert 'remaining_lease' in string to 'remaining_lease_months' in numerical

In [122]:
# Function to convert string to months
def convert_to_months(duration_str):
    split_parts = duration_str.split(' ')
    
    if len(split_parts) == 4:
        years = int(split_parts[0])
        months = int(split_parts[2])
        total_months = years * 12 + months
    
    else:
        years = int(split_parts[0])
        total_months = years * 12
        
    return total_months

# Apply the conversion function to the Duration column
df_hdb['remaining_lease_months']  = df_hdb['remaining_lease'].apply(convert_to_months)

In [123]:
#drop remaining_lease
df_hdb.drop('remaining_lease', axis=1, inplace=True)

In [124]:
df_hdb.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease_months
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,232000.0,736
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,250000.0,727
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,262000.0,749
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,265000.0,745
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,265000.0,749


In [125]:
X_df_hdb = df_hdb[['remaining_lease_months',
                    'floor_area_sqm',
                    'town',
                    'storey_range']]
X_df_hdb

,remaining_lease_months,floor_area_sqm,town,storey_range
0,736,44.0,ANG MO KIO,10 TO 12
1,727,67.0,ANG MO KIO,01 TO 03
2,749,67.0,ANG MO KIO,01 TO 03
3,745,68.0,ANG MO KIO,04 TO 06
4,749,67.0,ANG MO KIO,01 TO 03
...,...,...,...,...
154663,732,84.0,YISHUN,04 TO 06
154664,753,83.0,YISHUN,10 TO 12
154665,737,84.0,YISHUN,04 TO 06
154666,761,104.0,YISHUN,04 TO 06


In [126]:
y_df_hdb = df_hdb['resale_price']
y_df_hdb

0         232000.0
1         250000.0
2         262000.0
3         265000.0
4         265000.0
            ...   
154663    410000.0
154664    455000.0
154665    458000.0
154666    575000.0
154667    715000.0
Name: resale_price, Length: 154668, dtype: float64

In [127]:
# Encode categorical columns

In [128]:
categorical_columns = X_df_hdb.select_dtypes(include='object').columns.tolist()
categorical_columns

['town', 'storey_range']

In [129]:
X_df_hdb['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

In [130]:
town_mapping = {
    'ANG MO KIO': 0,
    'BEDOK': 1,
    'BISHAN': 2,
    'BUKIT BATOK': 3,
    'BUKIT MERAH': 4,
    'BUKIT PANJANG': 5,
    'BUKIT TIMAH': 6,
    'CENTRAL AREA': 7,
    'CHOA CHU KANG': 8,
    'CLEMENTI': 9,
    'GEYLANG': 10,
    'HOUGANG': 11,
    'JURONG EAST': 12,
    'JURONG WEST': 13,
    'KALLANG/WHAMPOA': 14,
    'MARINE PARADE': 15,
    'PASIR RIS': 16,
    'PUNGGOL': 17,
    'QUEENSTOWN': 18,
    'SEMBAWANG': 19,
    'SENGKANG': 20,
    'SERANGOON': 21,
    'TAMPINES': 22,
    'TOA PAYOH': 23,
    'WOODLANDS': 24,
    'YISHUN': 25
}


In [131]:
storey_mapping = {
    '01 TO 03': 0,
    '04 TO 06': 1,
    '07 TO 09': 2,
    '10 TO 12': 3,
    '13 TO 15': 4,
    '16 TO 18': 5,
    '19 TO 21': 6,
    '22 TO 24': 7,
    '25 TO 27': 8,
    '28 TO 30': 9,
    '31 TO 33': 10,
    '34 TO 36': 11,
    '37 TO 39': 12,
    '40 TO 42': 13,
    '43 TO 45': 14,
    '46 TO 48': 15,
    '49 TO 51': 16
}


In [132]:
X_df_hdb['town'] = X_df_hdb['town'].map(town_mapping)
X_df_hdb['storey_range'] = X_df_hdb['storey_range'].map(storey_mapping)
X_df_hdb.head()

C:\Users\Zach Chen\AppData\Local\Temp\ipykernel_11156\398897315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df_hdb['town'] = X_df_hdb['town'].map(town_mapping)
C:\Users\Zach Chen\AppData\Local\Temp\ipykernel_11156\398897315.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df_hdb['storey_range'] = X_df_hdb['storey_range'].map(storey_mapping)


,remaining_lease_months,floor_area_sqm,town,storey_range
0,736,44.0,0,3
1,727,67.0,0,0
2,749,67.0,0,0
3,745,68.0,0,1
4,749,67.0,0,0


In [133]:
# Scale all features

In [134]:
from sklearn.preprocessing import MinMaxScaler
import joblib

scaler = MinMaxScaler()
X_df_hdb_scaled_array = scaler.fit_transform(X_df_hdb)

joblib.dump(scaler, 'scaler.joblib')


['scaler.joblib']

In [135]:
X_df_hdb_scaled = pd.DataFrame(X_df_hdb_scaled_array, columns = X_df_hdb.columns)
X_df_hdb_scaled.head()

,remaining_lease_months,floor_area_sqm,town,storey_range
0,0.339879,0.059633,0.0,0.1875
1,0.326284,0.165138,0.0,0.0000
2,0.359517,0.165138,0.0,0.0000
3,0.353474,0.169725,0.0,0.0625
4,0.359517,0.165138,0.0,0.0000


In [136]:
# 5. Modelling Data

In [137]:
## Split into train and test sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df_hdb_scaled, y_df_hdb, test_size = 0.3, random_state=43)

In [138]:
# using ridge regression

In [139]:
from sklearn.linear_model import Ridge

In [140]:
ridge_model = Ridge()
ridge_model.fit(X_train, y_train)

Ridge()

In [141]:
y_pred_train = ridge_model.predict(X_train)
y_pred_test = ridge_model.predict(X_test)

In [142]:
print("train r2_score is:", round(r2_score(y_train, y_pred_train), 3))
print("test r2_score is:", round(r2_score(y_test, y_pred_test),3))

train r2_score is: 0.548
test r2_score is: 0.546


In [143]:
# deploy ridge_model

In [144]:
import joblib
joblib.dump(ridge_model, 'ridge_model.pkl')

['ridge_model.pkl']